## □ kNN 머신러닝 알고리즘을 이용하여 유방암 데이터 분류 데이터 분석
    1단계 : 데이터 수집
    2단계 : 데이터 탐색과 준비
    3단계 : 데이터 모델로 훈련
    4단계 : 모델 성능 평가
    5단계 : 모델 성능개선

### ■ 1단계 : 데이터 수집
    569개의 진단 데이터셋이며 암 조직 검사 예시
    32개의 특징을 갖고 있으며 디지털 이미지에 존재하는 세포핵의 특성을 나타냄
        반지름, 질감, 둘레, 넓이, 매끄러움, 조밀성, 오목함, 오목점, 대칭성, 프랙탈 차원
        정답(라벨) : diagnosis(진단) 양성(B) / 악성(M)

### ■ 2단계: 데이터 시각화

In [1]:
#1. 데이터를 로드 한다.
wbcd <- read.csv("wisc_bc_data.csv", header=T,  stringsAsFactors=FALSE)
table(wbcd$diagnosis)


  B   M 
357 212 

In [ ]:
#2. diagnosis 를 factor 로 변환한다
wbcd$diagnosis <- factor(wbcd$diagnosis,
                          levels =c("B","M"),
                   labels = c("Benign","Maliganant"))

#3. 데이터를 shuffle 시킨다.
wbcd_shuffle <- wbcd[sample(nrow(wbcd)), ]

#4. 데이터에서 id 를 제외 시킨다
wbcd2 <- wbcd_shuffle[-1]

#5. 데이터를 정규화 한다.
normalize <- function(x) {
      return ( (x-min(x)) / (max(x) - min(x))  )
}
wbcd_n  <- as.data.frame(lapply(wbcd2[2:31],normalize))

#6. train 데이터와 test 데이터로 9 대 1로 나눈다
train_num<-round(0.9*nrow(wbcd_n),0)
wbcd_train<-wbcd_n[1:train_num,]
wbcd_test<-wbcd_n[(train_num+1):nrow(wbcd_n),]

#7. train 데이터를 데이터와 라벨로 나누고 test  데이터를 데이터와 라벨로 나누시오 ~
wbcd_train_label <- wbcd2[1:train_num,1]
wbcd_test_label <- wbcd2[(train_num+1):nrow(wbcd_n),1]

#8. knn 모델로 훈련시켜서 모델을 만들고 바로 그 모델에 test 데이터를 넣어서 정확도를 확인한다
result1 <- knn(train=wbcd_train, test=wbcd_test, cl=wbcd_train_label, k=21)
result1